In [18]:
import pandas as pd
import numpy as np
import cv2 
import os
import glob

In [19]:
from tensorflow.keras.applications import ResNet50
incept_model=ResNet50(include_top=True)
#incept_model.summary()  

from keras.models import Model
last=incept_model.layers[-2].output
modele=Model(inputs = incept_model.input,outputs = last)
modele.summary()    

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 56, 56, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 56, 56, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 28, 28, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 28, 28, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 14, 14, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 14, 14, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 14, 14, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 14, 14, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 14, 14, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 7, 7, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

In [20]:
images_path="flickr30k/Images/flickr30k_images/"
images=glob.glob(images_path+"*.jpg")
caption_path = 'flickr30k/captions.txt'
with open(caption_path, 'r') as f:
    val_captions = f.readlines()
    
# Creating lists to store image paths and corresponding val_captions
val_image_paths = []
val_captions_dict = {}
val_images_features = {}
val_count = 0

for caption in val_captions[int(len(val_captions)*0.7):]:
    image_id,caption_text= caption.strip().split('.jpg,')
    image_path = os.path.join(images_path, image_id+'.jpg')
    if(image_path not in val_image_paths):
        val_image_paths.append(image_path)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        img = img.reshape(1,224,224,3)
        pred = modele.predict(img).reshape(2048,)  
        img_name = image_path.split('/')[-1]
        caption=caption_text.strip('"')
        if img_name not in val_captions_dict:
            val_captions_dict[img_name] = [caption]
        else:
            val_captions_dict[img_name].append(caption)
        val_images_features[img_name] = pred
    val_count += 1
    
    if val_count % 50 == 0:
        print(val_count)

1/1 [==============================] - 0s 52ms/step
50
1/1 [==============================] - 0s 52ms/step
100
1/1 [==============================] - 0s 53ms/step
150
1/1 [==============================] - 0s 56ms/step
200
1/1 [==============================] - 0s 54ms/step
250
1/1 [==============================] - 0s 54ms/step
300
1/1 [==============================] - 0s 54ms/step
350
1/1 [==============================] - 0s 56ms/step
400
1/1 [==============================] - 0s 56ms/step
450
1/1 [==============================] - 0s 57ms/step
500
1/1 [==============================] - 0s 115ms/step
550
1/1 [==============================] - 0s 61ms/step
600
1/1 [==============================] - 0s 62ms/step
650
1/1 [==============================] - 0s 61ms/step
700
1/1 [==============================] - 0s 64ms/step
750
1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 60ms/step
800
1/1 [==============================] - 0s 80ms/step
850
1/1 [==============================] - 0s 61ms/step
900
1/1 [==============================] - 0s 61ms/step
950
1/1 [==============================] - 0s 63ms/step
1000
1/1 [==============================] - 0s 62ms/step
1050
1/1 [==============================] - 0s 59ms/step
1100
1/1 [==============================] - 0s 62ms/step
1150
1/1 [==============================] - 0s 62ms/step
1200
1/1 [==============================] - 0s 62ms/step
1250
1/1 [==============================] - 0s 58ms/step
1300
1/1 [==============================] - 0s 60ms/step
1350
1/1 [==============================] - 0s 64ms/step
1400
1/1 [==============================] - 0s 65ms/step
1450
1/1 [==============================] - 0s 63ms/step
1500
1/1 [==============================] - 0s 62ms/step
1550
1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 60ms/step
1600
1/1 [==============================] - 0s 61ms/step
1650
1/1 [==============================] - 0s 61ms/step
1700
1/1 [==============================] - 0s 62ms/step
1750
1/1 [==============================] - 0s 65ms/step
1800
1/1 [==============================] - 0s 59ms/step
1850
1/1 [==============================] - 0s 61ms/step
1900
1/1 [==============================] - 0s 60ms/step
1950
1/1 [==============================] - 0s 62ms/step
2000
1/1 [==============================] - 0s 63ms/step
2050
1/1 [==============================] - 0s 63ms/step
2100
1/1 [==============================] - 0s 60ms/step
2150
1/1 [==============================] - 0s 59ms/step
2200
1/1 [==============================] - 0s 60ms/step
2250
1/1 [==============================] - 0s 60ms/step
2300
1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step
2350
1/1 [==============================] - 0s 60ms/step
2400
1/1 [==============================] - 0s 58ms/step
2450
1/1 [==============================] - 0s 62ms/step
2500
1/1 [==============================] - 0s 62ms/step
2550
1/1 [==============================] - 0s 61ms/step
2600
1/1 [==============================] - 0s 59ms/step
2650
1/1 [==============================] - 0s 59ms/step
2700
1/1 [==============================] - 0s 60ms/step
2750
1/1 [==============================] - 0s 60ms/step
2800
1/1 [==============================] - 0s 61ms/step
2850
1/1 [==============================] - 0s 62ms/step
2900
1/1 [==============================] - 0s 63ms/step
2950
1/1 [==============================] - 0s 61ms/step
3000
1/1 [==============================] - 0s 60ms/step
3050
1/1 [==============================] - 0s 61ms/step
3100
1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 59ms/step
3150
1/1 [==============================] - 0s 64ms/step
3200
1/1 [==============================] - 0s 64ms/step
3250
1/1 [==============================] - 0s 61ms/step
3300
1/1 [==============================] - 0s 64ms/step
3350
1/1 [==============================] - 0s 59ms/step
3400
1/1 [==============================] - 0s 58ms/step
3450
1/1 [==============================] - 0s 60ms/step
3500
1/1 [==============================] - 0s 63ms/step
3550
1/1 [==============================] - 0s 58ms/step
3600
1/1 [==============================] - 0s 57ms/step
3650
1/1 [==============================] - 0s 60ms/step
3700
1/1 [==============================] - 0s 59ms/step
3750
1/1 [==============================] - 0s 64ms/step
3800
1/1 [==============================] - 0s 59ms/step
3850
1/1 [==============================] - 0s 64ms/step


...

1/1 [==============================] - 0s 49ms/step
44250
1/1 [==============================] - 0s 49ms/step
44300
1/1 [==============================] - 0s 47ms/step
44350
1/1 [==============================] - 0s 60ms/step
44400
1/1 [==============================] - 0s 45ms/step
44450
1/1 [==============================] - 0s 48ms/step
44500
1/1 [==============================] - 0s 46ms/step
44550
1/1 [==============================] - 0s 48ms/step
44600
1/1 [==============================] - 0s 51ms/step
44650
1/1 [==============================] - 0s 49ms/step
44700
1/1 [==============================] - 0s 51ms/step
44750
1/1 [==============================] - 0s 48ms/step
44800
1/1 [==============================] - 0s 48ms/step
44850
1/1 [==============================] - 0s 49ms/step
44900
1/1 [==============================] - 0s 48ms/step
44950
1/1 [==============================] - 0s 49ms/step
45000
1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 47ms/step
45050
1/1 [==============================] - 0s 48ms/step
45100
1/1 [==============================] - 0s 49ms/step
45150
1/1 [==============================] - 0s 49ms/step
45200
1/1 [==============================] - 0s 48ms/step
45250
1/1 [==============================] - 0s 50ms/step
45300
1/1 [==============================] - 0s 48ms/step
45350
1/1 [==============================] - 0s 49ms/step
45400
1/1 [==============================] - 0s 49ms/step
45450
1/1 [==============================] - 0s 49ms/step
45500
1/1 [==============================] - 0s 50ms/step
45550
1/1 [==============================] - 0s 49ms/step
45600
1/1 [==============================] - 0s 48ms/step
45650
1/1 [==============================] - 0s 49ms/step
45700
1/1 [==============================] - 0s 47ms/step
45750
1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 50ms/step
45800
1/1 [==============================] - 0s 48ms/step
45850
1/1 [==============================] - 0s 49ms/step
45900
1/1 [==============================] - 0s 48ms/step
45950
1/1 [==============================] - 0s 49ms/step
46000
1/1 [==============================] - 0s 51ms/step
46050
1/1 [==============================] - 0s 56ms/step
46100
1/1 [==============================] - 0s 49ms/step
46150
1/1 [==============================] - 0s 49ms/step
46200
1/1 [==============================] - 0s 48ms/step
46250
1/1 [==============================] - 0s 50ms/step
46300
1/1 [==============================] - 0s 49ms/step
46350
1/1 [==============================] - 0s 48ms/step
46400
1/1 [==============================] - 0s 50ms/step
46450
1/1 [==============================] - 0s 50ms/step
46500
1/1 [==============================] - 0s 48ms/step
46550
1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 49ms/step
46600
1/1 [==============================] - 0s 50ms/step
46650
1/1 [==============================] - 0s 49ms/step
46700
1/1 [==============================] - 0s 50ms/step
46750
1/1 [==============================] - 0s 49ms/step
46800
1/1 [==============================] - 0s 50ms/step
46850
1/1 [==============================] - 0s 52ms/step
46900
1/1 [==============================] - 0s 48ms/step
46950
1/1 [==============================] - 0s 50ms/step
47000
1/1 [==============================] - 0s 49ms/step
47050
1/1 [==============================] - 0s 51ms/step
47100
1/1 [==============================] - 0s 49ms/step
47150
1/1 [==============================] - 0s 50ms/step
47200
1/1 [==============================] - 0s 51ms/step
47250
1/1 [==============================] - 0s 48ms/step
47300
1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 53ms/step
47350
1/1 [==============================] - 0s 51ms/step
47400
1/1 [==============================] - 0s 49ms/step
47450
1/1 [==============================] - 0s 47ms/step
47500
1/1 [==============================] - 0s 49ms/step
47550
1/1 [==============================] - 0s 49ms/step
47600
1/1 [==============================] - 0s 50ms/step
47650
1/1 [==============================] - 0s 48ms/step


In [21]:
def preprocessed(txt):
    modified = txt.lower()
    modified = 'startofseq ' + modified + ' endofseq'
    return modified

In [22]:
for k,v in val_captions_dict.items():
    for vv in v:
        val_captions_dict[k][v.index(vv)] = preprocessed(vv)

In [23]:
val_count_words = {}
for k,vv in val_captions_dict.items():
    for v in vv:
        for word in v.split():
            if word not in val_count_words:

                val_count_words[word] = 0

            else:
                val_count_words[word] += 1

In [24]:
THRESH = -1
val_count = 1
new_dict = {}
for k,v in val_count_words.items():
    if val_count_words[k] > THRESH:
        new_dict[k] = val_count
        val_count += 1
        

In [25]:
new_dict['<OUT>'] = len(new_dict) 
val_captions_backup = val_captions_dict.copy()
val_captions_dict = val_captions_backup.copy()
for k, vv in val_captions_dict.items():
    for v in vv:
        val_encoded = []
        for word in v.split():  
            if word not in new_dict:
                val_encoded.append(new_dict['<OUT>'])
            else:
                val_encoded.append(new_dict[word])


        val_captions_dict[k][vv.index(v)] = val_encoded

In [26]:
from tensorflow.keras.utils import to_categorical
from keras.utils import pad_sequences
MAX_LEN = 0
for k, vv in val_captions_dict.items():
    for v in vv:
        if len(v) > MAX_LEN:
            MAX_LEN = len(v)

In [27]:
Batch_size = 5000
VOCAB_SIZE = len(new_dict)

def generator(photo, caption):
    n_samples = 0
    
    X = []
    y_in = []
    y_out = []
    
    for k, vv in caption.items():
        for v in vv:
            for i in range(1, len(v)):
                X.append(photo[k])

                in_seq= [v[:i]]
                out_seq = v[i]

                in_seq = pad_sequences(in_seq, maxlen=MAX_LEN, padding='post', truncating='post')[0]
                out_seq = to_categorical([out_seq], num_classes=VOCAB_SIZE)[0]

                y_in.append(in_seq)
                y_out.append(out_seq)
            
    return X, y_in, y_out

In [29]:
X_val, y_in_val, y_out_val = generator(val_images_features, val_captions_dict)


In [30]:
import pickle
val_variables = {
    'X_val': X_val,
    'y_in_val': y_in_val,
    'y_out_val': y_out_val,
}

with open('val_variables.pkl', 'wb') as file:
    pickle.dump(val_variables, file)

In [31]:
with open('val_variables.pkl', 'rb') as file:
    loaded_val_variables = pickle.load(file)

In [32]:
X_val = np.array(val_variables['X_val'])
y_in_val = np.array(val_variables['y_in_val'], dtype='float64')
y_out_val = np.array(val_variables['y_out_val'], dtype='float64')

In [34]:
X_val.shape, y_in_val.shape, y_out_val.shape

((185540, 2048), (185540, 69), (185540, 7368))